# QdrantVectorStore

```{=mdx}
:::tip 兼容性
仅适用于 Node.js。
:::
```

[Qdrant](https://qdrant.tech/) 是一个向量相似性搜索引擎。它提供了一个可用于生产的便捷服务，具有方便的 API 来存储、搜索和管理点（即带有附加负载的向量）。

本指南提供了关于如何快速开始使用 Qdrant [向量存储](/docs/concepts/#vectorstores) 的简要概述。如需了解 `QdrantVectorStore` 所有功能和配置的详细文档，请参阅 [API 参考](https://api.js.langchain.com/classes/langchain_qdrant.QdrantVectorStore.html)。

## 概览

### 集成详情

| 类 | 包 | [Python 支持](https://python.langchain.com/docs/integrations/vectorstores/qdrant/) | 包的最新版本 |
| :--- | :--- | :---: | :---: |
| [`QdrantVectorStore`](https://api.js.langchain.com/classes/langchain_qdrant.QdrantVectorStore.html) | [`@langchain/qdrant`](https://npmjs.com/@langchain/qdrant) | ✅ |  ![NPM - 版本](https://img.shields.io/npm/v/@langchain/qdrant?style=flat-square&label=%20&) |

## 设置

要使用 Qdrant 向量存储，你需要设置一个 Qdrant 实例并安装 `@langchain/qdrant` 集成包。

本指南还将使用 [OpenAI 嵌入模型](/docs/integrations/text_embedding/openai)，这要求你安装 `@langchain/openai` 集成包。你也可以根据需要使用[其他支持的嵌入模型](/docs/integrations/text_embedding)。

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/qdrant @langchain/core @langchain/openai
</Npm2Yarn>
```

安装所需依赖后，按照 [Qdrant 设置指南](https://qdrant.tech/documentation/quickstart/) 使用 Docker 在你的计算机上运行一个 Qdrant 实例。注意你的容器运行的 URL。

### 凭证

完成上述操作后，设置一个 `QDRANT_URL` 环境变量：

```typescript
// 例如 http://localhost:6333
process.env.QDRANT_URL = "your-qdrant-url"
```

如果你在本指南中使用 OpenAI 嵌入模型，还需要设置你的 OpenAI 密钥：

```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
```

如果你想获得模型调用的自动追踪，也可以通过取消注释以下代码来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

```typescript
// process.env.LANGSMITH_TRACING="true"
// process.env.LANGSMITH_API_KEY="your-api-key"
```

## 实例化

In [1]:
import { QdrantVectorStore } from "@langchain/qdrant";
import { OpenAIEmbeddings } from "@langchain/openai";

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
});

const vectorStore = await QdrantVectorStore.fromExistingCollection(embeddings, {
  url: process.env.QDRANT_URL,
  collectionName: "langchainjs-testing",
});

## 管理向量存储

### 向向量存储中添加条目

In [3]:
import type { Document } from "@langchain/core/documents";

const document1: Document = {
  pageContent: "The powerhouse of the cell is the mitochondria",
  metadata: { source: "https://example.com" }
};

const document2: Document = {
  pageContent: "Buildings are made out of brick",
  metadata: { source: "https://example.com" }
};

const document3: Document = {
  pageContent: "Mitochondria are made out of lipids",
  metadata: { source: "https://example.com" }
};

const document4: Document = {
  pageContent: "The 2024 Olympics are in Paris",
  metadata: { source: "https://example.com" }
}

const documents = [document1, document2, document3, document4];

await vectorStore.addDocuments(documents);

顶级文档ID和删除当前不受支持。

## 查询向量存储

一旦创建了向量存储并添加了相关文档，您很可能会希望在链或代理运行期间查询它。

### 直接查询

执行一个简单的相似性搜索可以按如下方式进行：

In [7]:
const filter = {
  "must": [
      { "key": "metadata.source", "match": { "value": "https://example.com" } },
  ]
};

const similaritySearchResults = await vectorStore.similaritySearch("biology", 2, filter);

for (const doc of similaritySearchResults) {
  console.log(`* ${doc.pageContent} [${JSON.stringify(doc.metadata, null)}]`);
}

* The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* Mitochondria are made out of lipids [{"source":"https://example.com"}]


更多关于Qdrant过滤器语法的信息请参见[此页面](https://qdrant.tech/documentation/concepts/filtering/)。请注意，所有值都必须以 `metadata.` 开头。
如果你想执行相似性搜索并获得相应的分数，可以运行以下命令：

In [8]:
const similaritySearchWithScoreResults = await vectorStore.similaritySearchWithScore("biology", 2, filter)

for (const [doc, score] of similaritySearchWithScoreResults) {
  console.log(`* [SIM=${score.toFixed(3)}] ${doc.pageContent} [${JSON.stringify(doc.metadata)}]`);
}

* [SIM=0.165] The powerhouse of the cell is the mitochondria [{"source":"https://example.com"}]
* [SIM=0.148] Mitochondria are made out of lipids [{"source":"https://example.com"}]


### 通过转换为检索器进行查询

你还可以将向量存储转换为[检索器](/docs/concepts/retrievers)，以便在你的链中更方便地使用。

In [9]:
const retriever = vectorStore.asRetriever({
  // Optional filter
  filter: filter,
  k: 2,
});
await retriever.invoke("biology");

[
  Document {
    pageContent: 'The powerhouse of the cell is the mitochondria',
    metadata: { source: 'https://example.com' },
    id: undefined
  },
  Document {
    pageContent: 'Mitochondria are made out of lipids',
    metadata: { source: 'https://example.com' },
    id: undefined
  }
]


### 检索增强生成的用法

有关如何将此向量存储用于检索增强生成（RAG）的指南，请参阅以下部分：

- [教程：使用外部知识](/docs/tutorials/#working-with-external-knowledge)。
- [如何操作：使用RAG进行问答](/docs/how_to/#qa-with-rag)
- [检索概念文档](/docs/concepts/retrieval)

## API 参考文档

如需详细了解所有 `QdrantVectorStore` 功能和配置，请前往 [API 参考文档](https://api.js.langchain.com/classes/langchain_qdrant.QdrantVectorStore.html)。